# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

## Imports

In [874]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

Created fresh logs/timing_results.csv
Created fresh logs/timing.log


<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Retrieve Generated Test Data

In [875]:
prev_month_blobs = gpd.read_parquet('data/prev_month_blobs.parquet')
curr_month_blobs = gpd.read_parquet('data/curr_month_blobs.parquet')

In [896]:
# save to geojson to confirm in QGIS
prev_month_blobs.to_file('data/prev_month_blobs.geojson')
curr_month_blobs.to_file('data/curr_month_blobs.geojson')

In [876]:
prev_month_blobs.head()

,id,geohash,geometry
0,6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h,"POLYGON ((-81.10504 26.01885, -81.04984 26.018..."
1,a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq,"POLYGON ((-80.91675 26.04190, -80.86358 26.041..."
2,b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55,"POLYGON ((-80.73525 26.20304, -80.71903 26.203..."
3,d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s,"POLYGON ((-80.86288 26.17613, -80.79088 26.176..."
4,0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj,"POLYGON ((-81.28584 26.19397, -81.21442 26.193..."


In [877]:
prev_month_blobs = prev_month_blobs[['geometry', 'id', 'geohash']]
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.86288 26.17613, -80.79088 26.176...",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.28584 26.19397, -81.21442 26.193...",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


In [878]:
curr_month_blobs = curr_month_blobs[['geometry', 'id', 'geohash']]
curr_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.16204 27.42279, -81.06833 27.422...",8868a43b-d996-4161-bad3-ea183054f958,dhyk4n
1,"POLYGON ((-81.27418 27.57132, -81.25813 27.571...",1497d096-de50-4351-b33b-89904d705a8e,dhyjn9
2,"POLYGON ((-81.37152 27.86817, -81.31272 27.868...",741e60b6-255b-4031-8a98-54b1ef47b033,dhynv0
3,"POLYGON ((-81.74995 27.95319, -81.66345 27.953...",a8949814-3139-4a48-9aed-19922fc7806b,dhvzk9
4,"POLYGON ((-81.16827 28.08530, -81.09784 28.085...",5cb981af-0ab4-4087-87b4-0d8e622675d4,dhyrcy


In [879]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [880]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [881]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.86288 26.17613, -80.79088 26.176...",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.28584 26.19397, -81.21442 26.193...",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [882]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [883]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.1768 sec, CPU: 34.70%, Memory: 46.09MB
Function `convert_col_to_string` executed in 0.0982 sec, CPU: 3.60%, Memory: 7.62MB
                                            geometry  \
0  POLYGON ((-81.105035 26.018852, -81.049837 26....   
1  POLYGON ((-80.916747 26.041903, -80.863581 26....   
2  POLYGON ((-80.735253 26.20304, -80.719034 26.2...   
3  POLYGON ((-80.862883 26.176126, -80.790882 26....   
4  POLYGON ((-81.285842 26.193967, -81.214424 26....   

                                     id geohash  
0  6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0  dhwk5h  
1  a0fbead2-d8dc-4e05-b364-ab89d020fbfb  dhwkpq  
2  b4c64915-35eb-4d74-a211-ce41ad8f21f8  dhwt55  
3  d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3  dhwt0s  
4  0ce2ca44-0727-4364-978b-da23cd9cd784  dhwjpj  


/Users/sra/files/projects/matching_optimization/helpers.py:319: UserWarning: Geometry column does not contain geometry.
  list: A list of tuples representing the rows of the dataframe.
/Users/sra/files/projects/matching_optimization/helpers.py:319: UserWarning: Geometry column does not contain geometry.
  list: A list of tuples representing the rows of the dataframe.


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [884]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0052 sec, CPU: 40.00%, Memory: 0.00MB
Function `df_itertuple` executed in 0.0039 sec, CPU: 0.00%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [885]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.1154 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [886]:
prev_month_blobs.head(1)

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h


In [887]:
prev_month_blobs_wkt[0]

('POLYGON ((-81.105035 26.018852, -81.049837 26.018852, -81.049837 26.064626, -81.105035 26.064626, -81.105035 26.018852))',
 '6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0',
 'dhwk5h')

In [888]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.6406 sec, CPU: 36.00%, Memory: 43.38MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.5478 sec, CPU: 7.60%, Memory: 8.34MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [889]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0774 sec, CPU: 14.30%, Memory: 0.72MB
Retrieved 10000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0718 sec, CPU: 22.60%, Memory: 0.47MB


Compare the tables before and after for a sanity check:

In [890]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-81.105035 26.018852, -81.049837 26....",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.916747 26.041903, -80.863581 26....",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.735253 26.20304, -80.719034 26.2...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.862883 26.176126, -80.790882 26....",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.285842 26.193967, -81.214424 26....",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


In [891]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.86288 26.17613, -80.79088 26.176...",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.28584 26.19397, -81.21442 26.193...",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


Confirm that the original blob dfs and the retrieved dfs are the same:

In [892]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked! Now we can proceed to "matching" any "previous" polygons that overlap with the "current" polygons.

### 2. Matching

We will match the polygons using GeoPandas.

In [893]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Table matched_results created successfully.
Retrieved 10000 records from prev_blobs_wkt.
Function `match_geometries` executed in 0.0120 sec, CPU: 3.50%, Memory: 1.09MB
Function `match_geometries` executed in 0.0123 sec, CPU: 10.20%, Memory: 1.91MB
Function `match_geometries` executed in 0.0127 sec, CPU: 17.90%, Memory: 0.27MB
Function `match_geometries` executed in 0.0190 sec, CPU: 3.70%, Memory: 4.66MB
Function `match_geometries` executed in 0.0222 sec, CPU: 17.60%, Memory: 1.02MB
Function `match_geometries` executed in 0.0120 sec, CPU: 2.40%, Memory: 0.33MB
Function `match_geometries` executed in 0.0199 sec, CPU: 9.40%, Memory: 3.11MB
Function `match_geometries` executed in 0.0130 sec, CPU: 10.80%, Memory: 1.95MB
Function `match_geometries` executed in 0.0148 sec, CPU: 6.90%, Memory: 0.92MB
Function `match_geometries` executed in 0.0146 sec, CPU: 4.60%, Memory: 0.98MB
Function `match_geometries` executed in 0.0245 sec, CPU: 6.00%, Memory: 2.09MB
Function `run_parallel_matching` execu

In [894]:
# # testing logging for matching_geometries
# df_prev = h._retrieve_pg_table(postgresql_details, 'blob_matching', "prev_blobs_wkt", log_enabled=True)
# df_curr = h._retrieve_pg_table(postgresql_details, 'blob_matching', "curr_blobs_wkt", log_enabled=True)

# # Use a small subset for testing
# df_prev_sample = df_prev.sample(10, random_state=42)
# df_curr_sample = df_curr.sample(10, random_state=42)

# h.match_geometries(df_prev_sample, df_curr_sample)  # Check if this logs matches

## Optimized Approach

Having demonstrated a reasonable if not optimized way of matching polygons, we will now make use of geospatial-native tools to hopefully match polygons much more quickly.

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.